# NBA-22-2: Modèle de Régression (Score Exact)

**Objectif:** Prédire le score exact des deux équipes

**Challenge:** Plus difficile que la classification car variance élevée

**Objectif de performance:** MAE < 10 points

**Approche:** Deux modèles (un par équipe)

## 1. Setup

In [ ]:
import sys
sys.path.insert(0, '../src')

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

spark = SparkSession.builder.appName("NBA-Regression").getOrCreate()
print(f"Spark version: {spark.version}")

## 2. Chargement Données

In [ ]:
# Charger données
df = spark.read.parquet("../data/gold/ml_features")

# Vérifier les colonnes de score
print("Colonnes disponibles:")
for col_name in df.columns:
    if 'score' in col_name.lower():
        print(f"  - {col_name}")

df.select('game_id', 'score_home', 'score_away').show(5)

## 3. Features Spécifiques Régression

In [ ]:
# Features de base + features avancées pour régression
feature_cols = [
    # Features classification
    'win_pct_home',
    'win_pct_away',
    'win_pct_last_5_home',
    'win_pct_last_5_away',
    'avg_points_home',
    'avg_points_away',
    'rest_days_home',
    'rest_days_away',
    
    # Features additionnelles pour régression
    'pace_home',  # Rythme de jeu
    'pace_away',
    'offensive_rating_home',
    'offensive_rating_away',
    'defensive_rating_home',
    'defensive_rating_away',
    'effective_fg_pct_home',
    'effective_fg_pct_away',
    'turnover_rate_home',
    'turnover_rate_away',
]

print(f"Nombre de features: {len(feature_cols)}")

## 4. Split Train/Test

In [ ]:
# Split temporel
train_df = df.filter(df.season.isin(['2018-19', '2019-20', '2020-21', '2021-22']))
val_df = df.filter(df.season == '2022-23')
test_df = df.filter(df.season == '2023-24')

print(f"Train: {train_df.count()} matchs")
print(f"Validation: {val_df.count()} matchs")
print(f"Test: {test_df.count()} matchs")

# Stats des scores (pour baseline)
train_df.select('score_home', 'score_away').describe().show()

## 5. Baseline: Moyenne Saison

In [ ]:
# Baseline simple: prédire la moyenne
from pyspark.sql.functions import avg as spark_avg, abs as spark_abs

mean_scores = train_df.agg(
    spark_avg('score_home').alias('mean_home'),
    spark_avg('score_away').alias('mean_away')
).collect()[0]

mean_home = mean_scores['mean_home']
mean_away = mean_scores['mean_away']

print(f"Score moyen équipe à domicile: {mean_home:.1f}")
print(f"Score moyen équipe à l'extérieur: {mean_away:.1f}")

# Calculer MAE baseline sur test
baseline_df = test_df.withColumn('pred_home', F.lit(mean_home))
baseline_df = baseline_df.withColumn('pred_away', F.lit(mean_away))
baseline_df = baseline_df.withColumn('ae_home', spark_abs(F.col('score_home') - F.col('pred_home')))
baseline_df = baseline_df.withColumn('ae_away', spark_abs(F.col('score_away') - F.col('pred_away')))

baseline_mae = baseline_df.agg(
    spark_avg('ae_home').alias('mae_home'),
    spark_avg('ae_away').alias('mae_away')
).collect()[0]

print(f"\nBaseline MAE Home: {baseline_mae['mae_home']:.2f} points")
print(f"Baseline MAE Away: {baseline_mae['mae_away']:.2f} points")
print(f"Baseline MAE Moyen: {(baseline_mae['mae_home'] + baseline_mae['mae_away'])/2:.2f} points")

## 6. Modèles Random Forest

In [ ]:
# Assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

# Modèle pour équipe à domicile
rf_home = RandomForestRegressor(
    labelCol="score_home",
    featuresCol="features",
    numTrees=200,
    maxDepth=15,
    seed=42
)

# Modèle pour équipe à l'extérieur
rf_away = RandomForestRegressor(
    labelCol="score_away",
    featuresCol="features",
    numTrees=200,
    maxDepth=15,
    seed=42
)

# Pipelines
pipeline_home = Pipeline(stages=[assembler, rf_home])
pipeline_away = Pipeline(stages=[assembler, rf_away])

# Entraînement
print("Entraînement modèle HOME...")
model_home = pipeline_home.fit(train_df)

print("Entraînement modèle AWAY...")
model_away = pipeline_away.fit(train_df)

print("✅ Modèles entraînés")

## 7. Évaluation

In [ ]:
# Prédictions
pred_home = model_home.transform(test_df)
pred_away = model_away.transform(test_df)

# Évaluateur
evaluator = RegressionEvaluator()

# Métriques Home
mae_home = evaluator.evaluate(
    pred_home,
    {evaluator.labelCol: "score_home", evaluator.predictionCol: "prediction", evaluator.metricName: "mae"}
)
rmse_home = evaluator.evaluate(
    pred_home,
    {evaluator.labelCol: "score_home", evaluator.predictionCol: "prediction", evaluator.metricName: "rmse"}
)
r2_home = evaluator.evaluate(
    pred_home,
    {evaluator.labelCol: "score_home", evaluator.predictionCol: "prediction", evaluator.metricName: "r2"}
)

# Métriques Away
mae_away = evaluator.evaluate(
    pred_away,
    {evaluator.labelCol: "score_away", evaluator.predictionCol: "prediction", evaluator.metricName: "mae"}
)
rmse_away = evaluator.evaluate(
    pred_away,
    {evaluator.labelCol: "score_away", evaluator.predictionCol: "prediction", evaluator.metricName: "rmse"}
)
r2_away = evaluator.evaluate(
    pred_away,
    {evaluator.labelCol: "score_away", evaluator.predictionCol: "prediction", evaluator.metricName: "r2"}
)

# Moyennes
mae_avg = (mae_home + mae_away) / 2
rmse_avg = (rmse_home + rmse_away) / 2

print("\n=== RÉSULTATS ===")
print(f"Home - MAE: {mae_home:.2f}, RMSE: {rmse_home:.2f}, R²: {r2_home:.3f}")
print(f"Away - MAE: {mae_away:.2f}, RMSE: {rmse_away:.2f}, R²: {r2_away:.3f}")
print(f"\nMoyenne - MAE: {mae_avg:.2f}, RMSE: {rmse_avg:.2f}")
print(f"\nAmélioration vs Baseline: {(baseline_mae['mae_home'] + baseline_mae['mae_away'])/2 - mae_avg:.2f} points")

## 8. Visualisation des Erreurs

In [ ]:
# Convertir en pandas pour visualisation
pred_pd = pred_home.select('score_home', 'prediction').toPandas()
pred_pd.columns = ['actual', 'predicted']
pred_pd['error'] = pred_pd['actual'] - pred_pd['predicted']

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(pred_pd['actual'], pred_pd['predicted'], alpha=0.5)
axes[0].plot([80, 140], [80, 140], 'r--', lw=2)
axes[0].set_xlabel("Score Réel")
axes[0].set_ylabel("Score Prédit")
axes[0].set_title("Score Home: Réel vs Prédit")

# Distribution des erreurs
axes[1].hist(pred_pd['error'], bins=30, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--')
axes[1].set_xlabel("Erreur (points)")
axes[1].set_ylabel("Fréquence")
axes[1].set_title("Distribution des Erreurs - Home Team")

plt.tight_layout()
plt.show()

print(f"Erreur moyenne: {pred_pd['error'].mean():.2f} points")
print(f"Erreur std: {pred_pd['error'].std():.2f} points")

## 9. Analyse des Erreurs Élevées

In [ ]:
# Identifier les matchs avec erreur > 15 points
high_errors = pred_pd[abs(pred_pd['error']) > 15]
print(f"Matchs avec erreur > 15 pts: {len(high_errors)} ({len(high_errors)/len(pred_pd)*100:.1f}%)")

# Caractéristiques communes?
# TODO: Analyser ces matchs en détail

## 10. Sauvegarde

In [ ]:
# Sauvegarder les modèles
model_home.save("../models/regression_home")
model_away.save("../models/regression_away")

# Sauvegarder les métriques
import json
metrics = {
    'mae_home': mae_home,
    'mae_away': mae_away,
    'mae_avg': mae_avg,
    'rmse_home': rmse_home,
    'rmse_away': rmse_away,
    'r2_home': r2_home,
    'r2_away': r2_away,
    'baseline_mae': (baseline_mae['mae_home'] + baseline_mae['mae_away'])/2
}

with open("../models/regression_metrics.json", 'w') as f:
    json.dump(metrics, f, indent=2)

print("✅ Modèles et métriques sauvegardés")

## Résumé

**Résultats:**
- Baseline MAE: XX.XX points
- **Modèle MAE: XX.XX points** ← Objectif < 10 points
- Amélioration: XX.XX points

**Conclusion:** [À remplir]